# Bagging (nivel continente)

In [1]:
import pandas as pd
import sys
from pathlib import Path
import os
import json
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import STL


# adiciona a pasta src ao sys.path
sys.path.append(str(Path.cwd().parent.parent / "src")) # Necessário contar quantos niveis subir nas pastas, incluir 1 .parent para cada nivel.

# Importa o script paths de utils para geracão de caminhos absolutos
from utils.paths import resolve_env_path, PROJECT_ROOT
from classes.time_series_dataset import TimeSeriesDataset
from classes.plotter import Plotter
from utils.calculations import plot_stl_decomposition, process_df, block_bootstrap_stl_residuals, fit_sarima_on_simulations_long
from utils.model_metrics import rmse, mae, mape

# Adiciona caminhos importantes
ARRIVALS_PATH = resolve_env_path("ARRIVALS_PATH")

# Selecting seaborn theme
sns.set_theme(
    style="ticks",
    palette="pastel",
    context="notebook"
)

## Usando dados imputados

In [2]:
df_path = os.path.join(PROJECT_ROOT, "notebooks", "modeling", "ets_forecast_TRAIN.csv")
df = pd.read_csv(df_path)
df

,date,continent,actual,ets_forecast,error,abs_error,pct_error,ets_model,model_type,arima_model
0,2000-01-01,America_Do_Norte,22202.0,18082.070165,-4119.929835,4119.929835,-18.556571,"ETS(M,N,M)",AutoETS,NaN
1,2000-02-01,America_Do_Norte,19060.0,19986.541950,926.541950,926.541950,4.861185,"ETS(M,N,M)",AutoETS,NaN
2,2000-03-01,America_Do_Norte,17767.0,19325.610283,1558.610283,1558.610283,8.772501,"ETS(M,N,M)",AutoETS,NaN
3,2000-04-01,America_Do_Norte,14761.0,13363.934207,-1397.065793,1397.065793,-9.464574,"ETS(M,N,M)",AutoETS,NaN
4,2000-05-01,America_Do_Norte,12601.0,12939.110184,338.110184,338.110184,2.683201,"ETS(M,N,M)",AutoETS,NaN
...,...,...,...,...,...,...,...,...,...,...
2299,2023-08-01,TOTAL,80697.0,87601.714505,-6904.714505,6904.714505,-8.556346,"ETS(M,A,M)",HYBRID,"(1, 0, 0, 0, 1, 0, 0)"
2300,2023-09-01,TOTAL,82149.0,74041.895005,8107.104995,8107.104995,9.868781,"ETS(M,A,M)",HYBRID,"(1, 0, 0, 0, 1, 0, 0)"
2301,2023-10-01,TOTAL,92918.0,88634.817681,4283.182319,4283.182319,4.609637,"ETS(M,A,M)",HYBRID,"(1, 0, 0, 0, 1, 0, 0)"
2302,2023-11-01,TOTAL,114550.0,94906.380829,19643.619171,19643.619171,17.148511,"ETS(M,A,M)",HYBRID,"(1, 0, 0, 0, 1, 0, 0)"
